In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head(2)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis= 1, inplace = True)

#Drop noisy variables
application_df.drop(['SPECIAL_CONSIDERATIONS'], axis= 1, inplace = True)

In [3]:
app_counts = application_df["APPLICATION_TYPE"].value_counts()
replace_application = list(app_counts[app_counts < 100].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [4]:
class_counts = application_df["CLASSIFICATION"].value_counts()
replace_class = list(class_counts[class_counts < 1000].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_cats = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cats

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cats]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cats)

In [7]:
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cats,1)
application_df.head()

C:\Users\menon\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

C:\Users\menon\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2150      
                                                                 
 dense_1 (Dense)             (None, 10)                510       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 2,721
Trainable params: 2,721
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [13]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
  1/804 [..............................] - ETA: 21:05 - loss: 0.6841 - accuracy: 0.5938
Epoch 1: saving model to checkpoints\weights.01.hdf5
  5/804 [..............................] - ETA: 1:08 - loss: 0.6717 - accuracy: 0.5875 
Epoch 1: saving model to checkpoints\weights.01.hdf5
 10/804 [..............................] - ETA: 36s - loss: 0.6736 - accuracy: 0.5688 
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 20/804 [..............................] - ETA: 22s - loss: 0.6620 - accuracy: 0.6047
Epoch 1: saving model to checkpoints\weights.01.hdf5
 25/804 [..............................] - ETA: 20s - loss: 0.6583 - accuracy: 0.6112
Epoch 1: saving model to checkpoints\weights.01.hdf5
 30/804 [>.............................] - ETA: 18s - loss: 0.6552 - accuracy: 0.6198
Epoch 1: saving model to checkpoints\weights.01.hdf5
 35/804 [>.............................] - ETA: 17s - loss: 0.6511 - accuracy: 0.6304
Epoch 1: 

310/804 [==========>...................] - ETA: 7s - loss: 0.5833 - accuracy: 0.7108
Epoch 1: saving model to checkpoints\weights.01.hdf5
315/804 [==========>...................] - ETA: 7s - loss: 0.5840 - accuracy: 0.7100
Epoch 1: saving model to checkpoints\weights.01.hdf5
320/804 [==========>...................] - ETA: 7s - loss: 0.5846 - accuracy: 0.7097
Epoch 1: saving model to checkpoints\weights.01.hdf5
325/804 [===========>..................] - ETA: 7s - loss: 0.5843 - accuracy: 0.7097
Epoch 1: saving model to checkpoints\weights.01.hdf5
330/804 [===========>..................] - ETA: 7s - loss: 0.5841 - accuracy: 0.7099
Epoch 1: saving model to checkpoints\weights.01.hdf5
335/804 [===========>..................] - ETA: 6s - loss: 0.5833 - accuracy: 0.7105
Epoch 1: saving model to checkpoints\weights.01.hdf5
340/804 [===========>..................] - ETA: 6s - loss: 0.5824 - accuracy: 0.7113
Epoch 1: saving model to checkpoints\weights.01.hdf5
345/804 [===========>.............

610/804 [=====================>........] - ETA: 2s - loss: 0.5730 - accuracy: 0.7191
Epoch 1: saving model to checkpoints\weights.01.hdf5
615/804 [=====================>........] - ETA: 2s - loss: 0.5727 - accuracy: 0.7193
Epoch 1: saving model to checkpoints\weights.01.hdf5
620/804 [======================>.......] - ETA: 2s - loss: 0.5722 - accuracy: 0.7197
Epoch 1: saving model to checkpoints\weights.01.hdf5
625/804 [======================>.......] - ETA: 2s - loss: 0.5721 - accuracy: 0.7199
Epoch 1: saving model to checkpoints\weights.01.hdf5
630/804 [======================>.......] - ETA: 2s - loss: 0.5717 - accuracy: 0.7202
Epoch 1: saving model to checkpoints\weights.01.hdf5
635/804 [======================>.......] - ETA: 2s - loss: 0.5718 - accuracy: 0.7201
Epoch 1: saving model to checkpoints\weights.01.hdf5
640/804 [======================>.......] - ETA: 2s - loss: 0.5721 - accuracy: 0.7197
Epoch 1: saving model to checkpoints\weights.01.hdf5
645/804 [=======================>.

106/804 [==>...........................] - ETA: 9s - loss: 0.5499 - accuracy: 0.7361
Epoch 2: saving model to checkpoints\weights.02.hdf5
111/804 [===>..........................] - ETA: 9s - loss: 0.5496 - accuracy: 0.7368
Epoch 2: saving model to checkpoints\weights.02.hdf5
116/804 [===>..........................] - ETA: 9s - loss: 0.5504 - accuracy: 0.7360
Epoch 2: saving model to checkpoints\weights.02.hdf5
121/804 [===>..........................] - ETA: 9s - loss: 0.5518 - accuracy: 0.7332
Epoch 2: saving model to checkpoints\weights.02.hdf5
126/804 [===>..........................] - ETA: 9s - loss: 0.5529 - accuracy: 0.7326
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
136/804 [====>.........................] - ETA: 9s - loss: 0.5519 - accuracy: 0.7330
Epoch 2: saving model to checkpoints\weights.02.hdf5
141/804 [====>.........................] - ETA: 9s - loss: 0.5524 - accuracy: 0.7320
Epoch 2: saving model to checkpoi

416/804 [==============>...............] - ETA: 5s - loss: 0.5533 - accuracy: 0.7323
Epoch 2: saving model to checkpoints\weights.02.hdf5
421/804 [==============>...............] - ETA: 5s - loss: 0.5531 - accuracy: 0.7324
Epoch 2: saving model to checkpoints\weights.02.hdf5
426/804 [==============>...............] - ETA: 5s - loss: 0.5538 - accuracy: 0.7317
Epoch 2: saving model to checkpoints\weights.02.hdf5
431/804 [===============>..............] - ETA: 5s - loss: 0.5536 - accuracy: 0.7317
Epoch 2: saving model to checkpoints\weights.02.hdf5
436/804 [===============>..............] - ETA: 5s - loss: 0.5535 - accuracy: 0.7316
Epoch 2: saving model to checkpoints\weights.02.hdf5
441/804 [===============>..............] - ETA: 5s - loss: 0.5536 - accuracy: 0.7316
Epoch 2: saving model to checkpoints\weights.02.hdf5
446/804 [===============>..............] - ETA: 4s - loss: 0.5541 - accuracy: 0.7314
Epoch 2: saving model to checkpoints\weights.02.hdf5
451/804 [===============>.........

...

168/804 [=====>........................] - ETA: 8s - loss: 0.5265 - accuracy: 0.7487
Epoch 99: saving model to checkpoints\weights.99.hdf5
173/804 [=====>........................] - ETA: 8s - loss: 0.5245 - accuracy: 0.7504
Epoch 99: saving model to checkpoints\weights.99.hdf5
178/804 [=====>........................] - ETA: 8s - loss: 0.5243 - accuracy: 0.7504
Epoch 99: saving model to checkpoints\weights.99.hdf5
183/804 [=====>........................] - ETA: 7s - loss: 0.5249 - accuracy: 0.7495
Epoch 99: saving model to checkpoints\weights.99.hdf5
188/804 [======>.......................] - ETA: 7s - loss: 0.5245 - accuracy: 0.7493
Epoch 99: saving model to checkpoints\weights.99.hdf5

Epoch 99: saving model to checkpoints\weights.99.hdf5
198/804 [======>.......................] - ETA: 7s - loss: 0.5262 - accuracy: 0.7483
Epoch 99: saving model to checkpoints\weights.99.hdf5

Epoch 99: saving model to checkpoints\weights.99.hdf5
208/804 [======>.......................] - ETA: 7s - los

473/804 [================>.............] - ETA: 4s - loss: 0.5339 - accuracy: 0.7409
Epoch 99: saving model to checkpoints\weights.99.hdf5
479/804 [================>.............] - ETA: 4s - loss: 0.5343 - accuracy: 0.7404
Epoch 99: saving model to checkpoints\weights.99.hdf5
483/804 [=================>............] - ETA: 4s - loss: 0.5344 - accuracy: 0.7405
Epoch 99: saving model to checkpoints\weights.99.hdf5

Epoch 99: saving model to checkpoints\weights.99.hdf5
493/804 [=================>............] - ETA: 3s - loss: 0.5351 - accuracy: 0.7398
Epoch 99: saving model to checkpoints\weights.99.hdf5
498/804 [=================>............] - ETA: 3s - loss: 0.5348 - accuracy: 0.7403
Epoch 99: saving model to checkpoints\weights.99.hdf5
503/804 [=================>............] - ETA: 3s - loss: 0.5352 - accuracy: 0.7400
Epoch 99: saving model to checkpoints\weights.99.hdf5

Epoch 99: saving model to checkpoints\weights.99.hdf5
513/804 [==================>...........] - ETA: 3s - los

798/804 [============================>.] - ETA: 0s - loss: 0.5367 - accuracy: 0.7391
Epoch 99: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 10s 13ms/step - loss: 0.5369 - accuracy: 0.7388
Epoch 100/100
  1/804 [..............................] - ETA: 3s - loss: 0.5860 - accuracy: 0.6875
Epoch 100: saving model to checkpoints\weights.100.hdf5
  4/804 [..............................] - ETA: 17s - loss: 0.5796 - accuracy: 0.7109
Epoch 100: saving model to checkpoints\weights.100.hdf5
  9/804 [..............................] - ETA: 12s - loss: 0.5336 - accuracy: 0.7535
Epoch 100: saving model to checkpoints\weights.100.hdf5
 14/804 [..............................] - ETA: 11s - loss: 0.5578 - accuracy: 0.7232
Epoch 100: saving model to checkpoints\weights.100.hdf5

Epoch 100: saving model to checkpoints\weights.100.hdf5
 24/804 [..............................] - ETA: 9s - loss: 0.5356 - accuracy: 0.7370 
Epoch 100: saving model to checkpoints\weights

309/804 [==========>...................] - ETA: 5s - loss: 0.5364 - accuracy: 0.7406
Epoch 100: saving model to checkpoints\weights.100.hdf5
314/804 [==========>...................] - ETA: 5s - loss: 0.5371 - accuracy: 0.7403
Epoch 100: saving model to checkpoints\weights.100.hdf5
319/804 [==========>...................] - ETA: 5s - loss: 0.5374 - accuracy: 0.7401
Epoch 100: saving model to checkpoints\weights.100.hdf5
324/804 [===========>..................] - ETA: 5s - loss: 0.5367 - accuracy: 0.7406
Epoch 100: saving model to checkpoints\weights.100.hdf5
329/804 [===========>..................] - ETA: 5s - loss: 0.5359 - accuracy: 0.7409
Epoch 100: saving model to checkpoints\weights.100.hdf5
334/804 [===========>..................] - ETA: 5s - loss: 0.5365 - accuracy: 0.7404
Epoch 100: saving model to checkpoints\weights.100.hdf5
339/804 [===========>..................] - ETA: 5s - loss: 0.5371 - accuracy: 0.7393
Epoch 100: saving model to checkpoints\weights.100.hdf5
344/804 [====

609/804 [=====================>........] - ETA: 2s - loss: 0.5348 - accuracy: 0.7408
Epoch 100: saving model to checkpoints\weights.100.hdf5
614/804 [=====================>........] - ETA: 2s - loss: 0.5346 - accuracy: 0.7410
Epoch 100: saving model to checkpoints\weights.100.hdf5
619/804 [======================>.......] - ETA: 2s - loss: 0.5344 - accuracy: 0.7412
Epoch 100: saving model to checkpoints\weights.100.hdf5
624/804 [======================>.......] - ETA: 2s - loss: 0.5340 - accuracy: 0.7417
Epoch 100: saving model to checkpoints\weights.100.hdf5

Epoch 100: saving model to checkpoints\weights.100.hdf5
634/804 [======================>.......] - ETA: 2s - loss: 0.5341 - accuracy: 0.7416
Epoch 100: saving model to checkpoints\weights.100.hdf5
639/804 [======================>.......] - ETA: 2s - loss: 0.5339 - accuracy: 0.7419
Epoch 100: saving model to checkpoints\weights.100.hdf5
644/804 [=======================>......] - ETA: 1s - loss: 0.5335 - accuracy: 0.7421
Epoch 100: s

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("AlphabetSoupCharityOptimization.h5")